In [1]:
import numpy as np
import codecs
import array
import sys
import pandas as pd
import os
from datetime import datetime
import time

In [31]:
def load_tdms(path, ch_num):
    ch = [[] for _ in range(ch_num)]
    with codecs.open(path, 'rb') as f:
        while True:
            # リードインと呼ばれる部分の読み込み
            tdms = f.read(28) 
            # print(f.tell())
            # print(tdms)
            #if b'TDSm' != tdms[:4]:
            #    continue

            # ファイルを最後まで読み込んだら終了
            if tdms==b'': 
                break

            # データとその区切りを取得
            # 次のセグメントの位置を抽出
            seg_ofs = tdms[12:20]
            by1 = array.array('l')
            by1.frombytes( seg_ofs )
            seg_ofs = np.asarray(by1)[0]
            # データの位置を抽出
            data_ofs= tdms[20:28] 
            by2 = array.array('l')
            by2.frombytes( data_ofs )
            data_ofs = np.asarray(by2)[0]

            # データのある部分まで読み飛ばす
            tdms = f.read(data_ofs)
            #print(tdms[:200], len(tdms))
            #if len(ch[0])==2:
            #    exit()
            #if len(ch[0])==0:
            #    head=tdms
            # データ部分の読み込み
            tdms = f.read(seg_ofs-data_ofs)
            by = array.array('f')
            by.frombytes( tdms )
            data = np.asarray( by )
            # print(data.shape)

            # 各チャンネルを取得
            for i in range(ch_num):
                ch[i].append( data[i::ch_num].reshape(-1,1) )

    for i in range(ch_num):
        ch[i] = np.vstack(ch[i])[:,0]

    return ch

In [5]:
path = r'C:\Users\pmg07\Desktop\2021_1115_切削実験\dyanano\Dynamometer_26.tdms'

for_time_d = load_tdms(path, ch_num=3)

time_length = 16.668

# 時間を表す配列を作る
time_d = []
# 1サンプリングするのにかかる時間
dt = 1/(10**6)*3

# 時間を表す配列を作るために、16,548,822(flatten_all_dataの長さ)分だけ
# sample_rate × i をする。ただし、16.668(time_length)を超えない時まで
for i in range(1, len(for_time_d[0])+1):
    if(time_length < dt*(i-1)):
        break
    time_d.append(dt*(i-1))

print(len(time_d))

5215536


In [33]:
start=datetime.now()

last_t = 0

for i in range(25, 506, 2):
    nc_d_path = fr'C:\Users\pmg07\Desktop\2021_1115_切削実験\dyanano\Dynamometer_{i}.tdms'
    c_d_path = fr'C:\Users\pmg07\Desktop\2021_1115_切削実験\dyanano\Dynamometer_{i+1}.tdms'
    
    nc_d = load_tdms(nc_d_path, ch_num=3)
    c_d = load_tdms(c_d_path, ch_num=3)

    N = len(c_d[0])
    t_d = np.linspace(last_t, last_t + N*dt, N)  # 周波数軸　linspace(開始,終了,分割数)
    cd_d = t_d*(190/60)/1000

    nc_x_a = sum(nc_d[0]) / len(nc_d[0])
    nc_y_a = sum(nc_d[1]) / len(nc_d[1])
    nc_z_a = sum(nc_d[2]) / len(nc_d[2])

    x_g = 0.00895
    y_g = 0.00515
    z_g = 0.00804
    
    c_d[0][:] = (c_d[0][:] - nc_x_a)/ x_g 
    c_d[1][:] = (c_d[1][:] - nc_y_a)/ y_g 
    c_d[2][:] = (c_d[2][:] - nc_z_a)/ z_g 
    
#     dict1 = dict(time=time_d[:3000000], x=c_d[0][:3000000], y=c_d[1][:3000000], z=c_d[2][:3000000])
        
#     dict2 = dict(time=time_d[3000000:5215536-300000], x=c_d[0][3000000:5215536-300000], y=c_d[1][3000000:5215536-300000], z=c_d[2][3000000:5215536-300000])
    
    
#     dict1 = dict(time=t_d[:3000000], cd_d = cd_d[:3000000], x=c_d[0][:3000000], y=c_d[1][:3000000], z=c_d[2][:3000000])
#     dict2 = dict(time=t_d[3000000:], cd_d = cd_d[3000000:], x=c_d[0][3000000:], y=c_d[1][3000000:], z=c_d[2][3000000:])
        
    cd_d[0:-1]
    
    dict1 = dict(time=t_d[0:-1:10000], cd_d = cd_d[0:-1:10000], x=c_d[0][0:-1:10000], y=c_d[1][0:-1:10000], z=c_d[2][0:-1:10000])

    
    
    
#     print(len(t_d))
#     print(len(cd_d))
#     print(len(c_d[0]))
#     print(len(c_d[1]))
#     print(len(c_d[2]))
    df1 = pd.DataFrame(data=dict1)
#     df2 = pd.DataFrame(data=dict2)
    df1.to_csv(f'cuttingForce_10000_{i+1}.csv')
#     df2.to_csv(f'cuttingForce_practice_cd_d{i+1}.csv', mode='a', header=False)
    
    last_t = t_d[-1]
    print(i)
    print(t_d[-1])
    print(cd_d[-1])

print(datetime.now() - start)

25
15.646608
0.049547592
27
31.244586
0.098941189
29
46.83627
0.148314855
31
62.448527999999996
0.19775367199999996
33
78.063306
0.24720046899999998
35
93.671565
0.2966266225
37
109.28839500000001
0.3460799175
39
124.92184800000001
0.39558585200000007
41
140.55670800000001
0.44509624200000003
43
156.196008
0.49462069200000003
45
171.835257
0.5441449804999999
47
187.45237200000003
0.5935991780000001
49
203.07524100000003
0.6430715965
51
218.74071900000004
0.6926789435
53
234.38237100000003
0.7422108415000002
55
250.01922600000003
0.7917275490000001
57
265.66579500000006
0.8412750175000001
59
281.29842600000006
0.8907783490000002
61
296.94141300000007
0.9403144745000002
63
312.5845560000001
0.9898510940000003
65
328.2170670000001
1.0393540455
67
343.8334500000001
1.0888059250000002
69
359.4468720000001
1.1382484280000003
71
375.0571290000001
1.1876809085000002
73
390.6797790000001
1.2371526335000003
75
406.3116900000001
1.2866536850000003
77
421.9528650000001
1.3361840725000003
79
437.58

441
3268.2620700000025
10.349496555000007
443
3283.8745830000025
10.398936179500009
445
3299.4600750000027
10.448290237500009
447
3315.0437580000025
10.497638567000008
449
3330.6589380000023
10.547086637000008
451
3346.280805000002
10.596555882500006
453
3361.897827000002
10.646009785500006
455
3377.529096000002
10.695508804000006
457
3393.155646000002
10.744992879000005
459
3408.789081000002
10.794498756500007
461
3424.4220390000023
10.844003123500006
463
3440.0482470000024
10.893486115500007
465
3455.6794680000025
10.942984982000008
467
3471.3091680000025
10.992479032000007
469
3486.9356130000024
11.041962774500007
471
3502.5655860000024
11.091457689000007
473
3518.1933930000023
11.140945744500007
475
3533.8171710000024
11.190421041500008
477
3549.448023000002
11.239918739500006
479
3565.082814000002
11.289428911000007
481
3580.714947000002
11.338930665500007
483
3596.347305000002
11.388433132500007
485
3611.979642000002
11.437935533000005
487
3627.610155000002
11.487432157500006
489

In [3]:
nc_d_path = r'C:\Users\pmg07\Desktop\2021_1115_切削実験\dyanano\Dynamometer_25.tdms'
c_d_path = r'C:\Users\pmg07\Desktop\2021_1115_切削実験\dyanano\Dynamometer_26.tdms'

nc_d = load_tdms(nc_d_path, ch_num=3)
c_d = load_tdms(c_d_path, ch_num=3)

In [24]:
N = len(c_d[0])
print(N)
t_d = np.linspace(0, N*dt, N)  # 周波数軸　linspace(開始,終了,分割数)
print(t_d[-1])
# print(type(t_d))
# cd_d = np.linspace(0, t_d*(190/60)/1000, N)
cd_d = t_d*(190/60)/1000
print(cd_d[-1]*1000)

5215536
15.646608
49.547592
